In [1]:
import os
from sqlalchemy import create_engine


# Construct the connection URL
db_url = f"sqlite:///{os.path.abspath('/home/muregz/data_analysis/song_sales/chinook.db')}"

# Set the DATABASE_URL environment variable
os.environ['DATABASE_URL'] = db_url

# Create a database engine
engine = create_engine(os.environ['DATABASE_URL'])


In [2]:
#configuring mysql into jupyter notebook
%load_ext sql
%config SqlMagic.autopandas=True

1.Which tracks generated the most revenue? which album? which genre?

In [3]:
%%sql


UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?


2.Which tracks appeared in the most playlists?How many playlists did they appear in?

In [9]:
%%sql
SELECT tracks.Name, COUNT(*) AS num_appearances
FROM playlist_track
JOIN tracks
ON playlist_track.TrackId = tracks.TrackId
GROUP BY playlist_track.TrackId
ORDER BY COUNT(*) DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Name,num_appearances
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
5,Solomon HWV 67: The Arrival of the Queen of Sheba,5
6,Symphony No.5 in C Minor: I. Allegro con brio,5
7,Ave Maria,5
8,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",5
9,Die Walküre: The Ride of the Valkyries,5
